In [1]:
from joblib import load
import os
import json
import numpy as np
from pathlib import Path
import os
import pandas as pd

In [2]:
curation_root = Path("/mnt/i/bio/mave_curation/")
dataset_paths = list(map(lambda pth: pth.parent, curation_root.glob("*/observations.pkl")))

In [5]:
import logging
def read_dataset(path,return_counts=False):
    observations = load(path / "observations.pkl")
    hgvs_pro = load(path / "hgvs_pro.pkl")
    for k, v in hgvs_pro.items():
        if len(v) > len(set(v)):
            logging.warning(f"Duplicate HGVS_pro in {path} - {k} - n duplicates {len(v) - len(set(v))}")
    if return_counts:
        return {k : len(v) for k,v in observations.items()}
    return observations, hgvs_pro

In [7]:
stats = {str(pth.stem.replace("_"," - ")) : read_dataset(pth,return_counts=True) for pth in dataset_paths}
df = pd.DataFrame(stats).T
df.to_excel("dataset_stats.xlsx")
df

,p_lp,b_lb,vus,gnomad,synonymous,nonsynonymous
Adamovich - BRCA1 - Cisplatin,49,25,125,113,0,1359
Adamovich - BRCA1 - HDR,45,33,195,152,0,2150
Boettcher - TP53,181,102,608,98,0,7447
Erwood - BRCA2 - HEK293T,5,2,30,60,48,265
Erwood - NPC1 - HEK293T,28,1,41,15,103,637
Erwood - NPC1 - RPE1,1,0,4,3,10,63
Findlay - BRCA1 - SGE,164,71,397,393,295,1837
Findlay - VHL - SGE,134,2,309,9,180,1041
Gebbia - CHK2,5,4,1390,894,473,7955
Giacomelli - p53 - nullEtoposide,181,102,608,98,378,7487


In [8]:
datasets = {str(pth.stem.replace("_"," - ")) : read_dataset(pth,return_counts=False) for pth in dataset_paths}

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
nsamples = 6
fig,axs = plt.subplots(len(datasets),nsamples,figsize=(nsamples*7,7 * len(datasets)),sharex='row')
for ax,(ds_name, data) in zip(axs,datasets.items()):
    nsamples = len(data[0])
    for i, (k,v) in enumerate(data[0].items()):
        ax[i].hist(v,label=f"n={len(v)}")
        ax[i].legend()
        ax[i].set_title(f"Dataset: {ds_name}\nSample: {k}")

In [19]:
fig.savefig("dataset_histograms.pdf",bbox_inches='tight',format='pdf')

In [6]:
import json
with open("/mnt/i/bio/mave_curation/dataset_configs.json") as f:
    dataset_configs = json.load(f)

In [8]:
list(set([pth.stem for pth in dataset_paths]) - set(dataset_configs.keys()))

['Mighell_PTEN_lipidPhosphataseAssay', 'Matreyek_TPMT_VampSeq']